In [1]:
# importing all the required modules

from importlib import reload
import nltk
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
import os
import errno
import string
from nltk.corpus import reuters
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk import FreqDist
from nltk.text import TextCollection
import collections
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from prettytable import PrettyTable
import time
from datetime import timedelta
from sklearn.metrics import recall_score,precision_score,average_precision_score,f1_score,accuracy_score,roc_auc_score
from sklearn.preprocessing import label_binarize
from imblearn.metrics import geometric_mean_score
from sklearn import metrics
from sklearn.metrics.cluster import homogeneity_score,completeness_score
import statistics
import math
import sklearn.metrics 
from sklearn.model_selection import KFold
import csv
from sklearn.model_selection import train_test_split
from collections import Counter
import math
from scipy.spatial import distance
import statistics
from scipy.stats import pearsonr,entropy
from nltk.stem import WordNetLemmatizer
nltk.download('reuters')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold
from sklearn import svm, datasets
from sklearn.datasets import fetch_20newsgroups
from sklearn.metrics import pairwise_distances

def loadWebKbData(path, documents,labels):
    print(path)
    for root, dirs, files in os.walk(path):  
        for filename in files:
            try:
                #print root
                name = os.path.join(root, filename)
                #print name
                end=len(name)-len(filename)
                test=name[len(path)+1:end]
                for i in range(0,len(test)):
                    if test[i]=='\\':
                        labels.append(test[0:i])
                        break
                f = open(name, "rb").read()
                f=f.decode('ISO-8859-1', 'ignore')
                content=str(f)
                rawData.append(f)
                soup = BeautifulSoup(content)
                content=soup.get_text()
                documents.append(content)  
            except IOError as exc:
                if exc.errno != errno.EISDIR:
                       raise
'''
*****************************************Distnace measures*******************************************

'''
def Manhattan(doc1,doc2):
    return -1*distance.cityblock(doc1,doc2)                       
def Euclidean(a, b):#distance
    return -1*distance.euclidean(a,b)
def Cosine(a, b):#distance
    return 1-distance.cosine(a,b)
def Jaccard(a, b):#distance
    return 1-distance.jaccard(a,b)
def EnhancedJaccard(a, b):#distance
    a=[0 if x==0 else 1 for x in a]
    b=[0 if x==0 else 1 for x in b]
    return 1-distance.jaccard(a,b)
def KL(a, b):
    return 1-entropy(a,b)
def PCC(a, b):
    pcc, col=pearsonr(a,b)
    return abs(pcc)
def extendedJaccard(a,b):
    vector1=[0 if x==0 else 1 for x in a]
    vector2=[0 if x==0 else 1 for x in b]
    dot=np.dot(vector1,vector2)
    sum1=np.sum(vector1)
    sum2=np.sum(vector2)
    denom=math.sqrt(sum1)+math.sqrt(sum2)-dot
    if(denom!=0):
        return  (float(dot)/(denom))
    else:
        return -1
def bhatta(a,b):

    length=len(a)
    score = 0;
    score=np.sum(np.sqrt( np.multiply(a,b) ))
    distance=-1*np.log(score)
    return 1-distance;        

def JS(a, b):
   # normalize
    p = a/ np.sum(a)
    q = b/ np.sum(b)
    m = (p + q) / 2
    return 1-(entropy(p, m) + entropy(q, m)) / 2

def Pairwise(a,b):
    percentage=1 #100 percent
    K1=np.count_nonzero(a)
    K2=np.count_nonzero(b)
    k=percentage*min(K1,K2)
   # setA=set((-a).argsort()[:k])
   # setB=set((-b).argsort()[:k])
    
    setA=set(np.argpartition(a, len(a) - k)[-k:])
    setB=set(np.argpartition(b, len(b) - k)[-k:])
    union=setA.union(setB)
    elementsA=[a[ind] for ind in union]
    elementsB=[b[ind] for ind in union]
    dist= distance.cosine(elementsA, elementsB)
    return 1-dist

def Dice(a, b):
    a=[(a.astype(bool)).astype(int)]
    b=[(b.astype(bool)).astype(int)]
    return 1-distance.dice(a,b)
def Extended_Dice(a, b):
    a=[(a.astype(bool)).astype(int)]
    b=[(b.astype(bool)).astype(int)]
    sim=(2*np.dot(a,b))/(sum(a)**2 + sum(b)**2)
    return sim
def simIT(a,b,termOcc,docOcc):
    p1= np.divide(a,termOcc) 
    p2= np.divide(b,termOcc)
    minVal=np.minimum(p1,p2)
    pi=np.log((np.array(docOcc)/totalDocs))
    sIT=(2*np.sum(np.multiply(minVal,pi)))/((np.sum(np.multiply(p1,pi)))+(np.sum(np.multiply(p2,pi))))
    return (sIT)
def SP(doc1,doc2,N,allData):
    
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    nonzeroTerm=list(a.intersection(b))
    minArr=np.minimum(doc1[nonzeroTerm],doc2[nonzeroTerm])
    maxArr=np.maximum(doc1[nonzeroTerm],doc2[nonzeroTerm])
    ln=len(nonzeroTerm)
    docCount=np.zeros(ln)
    for ti in range(0,ln):
        tData=pd.Series(allData[:,ti])
        docCount[ti]=count_values_in_range(tData, minArr[ti], maxArr[ti])
    normFactor= len(a.union(b))
    dCount=np.array(docCount[np.nonzero(docCount)[0]])
    SP_val=0
    if(normFactor!=0):
        SP_val=sum(np.log(totalDocs/dCount))
        SP_val = (SP_val/normFactor)
    return SP_val

def PDSM1(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    intersection=sum(np.minimum(doc1,doc2))
    union=sum(np.maximum(doc1,doc2))
    return(intersection/union)
    return psdm
def PDSM(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    intersection=sum(np.minimum(doc1,doc2))
    union=sum(np.maximum(doc1,doc2))
    PF=len(a.intersection(b))
    M=len(doc1)
    AF=len(set(range(0,M))-(a.union(b)))
    psdm=(intersection/union)*((PF+1)/(M-AF+1))
    return psdm

def smtp(doc1,doc2,var):
    lemda=1
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    intersection=a.intersection(b) # indices where both docs have non zero elements
    union=a.union(b) # indices where either doc has non zero elements
    d1=np.array(list(a-intersection)) # doc1 !=0 and doc2=0
    d2=np.array(list(b-intersection)) # doc1 =0 and doc2!=0
    doc1=np.array(doc1)
    doc2=np.array(doc2)
    Nstar=0
    intersection=np.array(list(intersection))
    if (len(intersection)>0):
        term1=np.exp(-1*np.square(( doc1[intersection]-doc2[intersection] )/var[intersection]))
        Nstar=sum(0.5* (1+term1)) +lemda* -1 *(len(d1)+len(d2))
    else:
        Nstar=lemda* -1 *(len(d1)+len(d2))   
    Nunion=len(intersection)+len(d1)+len(d2)
    smtp=((Nstar/Nunion)+lemda)/(1+lemda)
    return smtp
def ISC(doc1,doc2): 
    dot=sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return isc

def NSMT(doc1,doc2):
    Dij=Nij=Di=Ni=Nj=Dj=0
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    intersection=a.intersection(b)# indices where both docs have non zero elements
    a=np.array(list(a-intersection)) 
    b=np.array(list(b-intersection))
    intersection=np.array(list(intersection))
    if len(intersection)>0:
      #  prod=np.multiply(doc1[intersection],doc2[intersection])
        Dij=np.dot(doc1[intersection],doc2[intersection])#sum(prod)
    if len(a)>0:
        Di=sum(doc1[a])
    if len(b)>0:
        Dj=sum(doc2[b])
    Nij=len(intersection)
    Ni=len(a)
    Nj=len(b)
    Nsmt=(Nij*Dij)/(Ni*Di+Dj*Nj)
    return Nsmt
def CSMB(doc1,doc2,alpha=0.5,Beta=0.5):
    sim1=CSM_P1(doc1,doc2)
    sim2=CSM_P2(doc1,doc2)
    simValue=alpha*sim1+Beta*sim2
    return simValue
def CSMB_MinMax(doc1,doc2,alpha=0.5,Beta=0.5):
    sim1=CSM_P1(doc1,doc2)
    sim2=CSM_P2(doc1,doc2)
    simValue=alpha*max(sim1,sim2)+Beta*min(sim1,sim2)
    return simValue
def BLAB_SM(doc1,doc2):
    sim1=0
    if dataset=="reuters":
        sim1=CSMB_MinMax(doc1,doc2,alpha=0.7,Beta=0.3)#CSMB12
    else:
        sim1=CSMB(doc1,doc2,alpha=0.9,Beta=0.1)# CSMB10
    return sim1

def ZSM(doc1,doc2):
    Dij=Nij=Di=Ni=Nj=Dj=0
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    intersection=a.intersection(b)# indices where both docs have non zero elements
    intersection=np.array(list(intersection))
    if len(intersection)>0:
      #  prod=np.multiply(doc1[intersection],doc2[intersection])
        Dij=np.dot(doc1[intersection],doc2[intersection])#sum(prod)
    Nij=len(intersection)
    zsm=(Nij*Dij)/(np.sum(doc1)*np.sum(doc2))
    return zsm
def CSM_P1(doc1,doc2):
    if ((not np.any(doc1)) or (not np.any( doc2))):   
        simValue=0  
    else:
        simValue=0
        N=len(doc1)#total_features;
        a=set(np.nonzero(doc1)[0])  
        b=set(np.nonzero(doc2)[0])   
        Nab=len(a.intersection(b))
        F=len(a.union(b))-Nab
        sim1=(1-F/N)
    return sim1
def CSM_P2(doc1,doc2):
    if ((not np.any(doc1)) or (not np.any( doc2))):   
        simValue=0  
    else:
        simValue=0
        a=set(np.nonzero(doc1)[0])  
        b=set(np.nonzero(doc2)[0])   
        Na=len(a)
        Nb=len(b)
        Nab=len(a.intersection(b))
        sim2=(2*Nab)/(Na+Nb)
    return sim2
def DDSMa(doc1,doc2):
    return 1-(np.sum(np.abs(doc1-doc2))/np.sum(doc1+doc2))
def DDSMb(doc1,doc2):
    return 1-(np.sum(np.square(doc1-doc2))/np.sum(np.square(doc1+doc2)))
def DDSMc(doc1,doc2):
    doc3=np.square(doc1)
    doc4=np.square(doc2)
    return 1-(np.sum(np.abs(doc3-doc4))/np.sum(np.square(doc1+doc2)))
def EN1_DDSMa(doc1,doc2):
    return DDSMa(doc1,doc2)*CSM_P1(doc1,doc2)
def EN_DDSMb(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  
    b=set(np.nonzero(doc2)[0])
    SF=len(list(a.intersection(b)))
    N=len(doc1)
    return DDSMb(doc1,doc2)*(SF+1)/(N+1)
def EN_DDSMc(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  
    b=set(np.nonzero(doc2)[0])
    SF=len(list(a.intersection(b)))
    N=len(doc1)
    return DDSMc(doc1,doc2)*(SF+1)/(N+1)
def BASM(doc1,doc2):
    return CSM_P2(doc1,doc2) 

def ESTB_SM(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  
    b=set(np.nonzero(doc2)[0])
    intersection=np.array(list(a.intersection(b)))
    comp1=np.array(list(a-b))
    comp2=np.array(list(b-a))
    X,Y,D1,D2,sim=0,0,0,0,0                   
    if len(intersection)>0:
        X=np.sum(doc1[intersection])
        Y=np.sum(doc2[intersection])
        if len(comp1)>0:
            D1=np.sum(doc1[comp1])
        if len(comp2)>0:
            D2=np.sum(doc2[comp2])
        if len(intersection)>0:
            sim=1/(1+((D1/X)+(D2/Y)))
    return sim 
def ESTBSM_Sim2(doc1,doc2):
    return ESTB_SM(doc1,doc2)*CSM_P2(doc1,doc2)
def pdsm1_Sim2(doc1,doc2):
    return PDSM1(doc1,doc2)*CSM_P2(doc1,doc2)
def mstbSM_Sim2(doc1,doc2):
    return MSTB_SM(doc1,doc2)*CSM_P2(doc1,doc2)

def ESTB_V1(doc1,doc2):
    return ESTB_SM(doc1,doc2)+CSM_P2(doc1,doc2)
def ESTB_V2(doc1,doc2):
    return ESTB_SM(doc1,doc2)*CSM_P2(doc1,doc2)*ANSM(doc1,doc2)
def ESTB_V3(doc1,doc2):
    return ((ESTB_SM(doc1,doc2)+ANSM(doc1,doc2))/2)*CSM_P2(doc1,doc2)

def EMX1(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  
    b=set(np.nonzero(doc2)[0])
    intersection=a.intersection(b)
    sim=0
    if len(intersection)>0:
        meanDiff=abs(np.mean(doc1)-np.mean(doc2))
        sim=1/(1+math.exp(-1*meanDiff))
    return sim
def EMX2(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  
    b=set(np.nonzero(doc2)[0])
    intersection=a.intersection(b)
    sim=0
    if len(intersection)>0:
        meanDiff=abs(np.mean(doc1)-np.mean(doc2))
        stdDiff=abs(np.std(doc1)-np.std(doc2))
        sim=1/(1+math.exp(-1*meanDiff*stdDiff))
    return sim 

def EMX7(doc1,doc2):
    sim=EMX1(doc1,doc2)
    if sim>0:
        sim=sim*ESTB_SM(doc1,doc2)*CSM_P2(doc1,doc2)
    return sim 
def EMX13(doc1,doc2):
    sim=EMX2(doc1,doc2)
    if sim>0:
        sim=sim*ESTB_SM(doc1,doc2)*CSM_P2(doc1,doc2)
    return sim 
def PCC_Sim2(doc1,doc2):#distance
    return PCC(doc1,doc2)*CSM_P2(doc1,doc2)
def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return isc
def EISC_Sim2(doc1,doc2):
    return EISC(doc1,doc2)*CSM_P2(doc1,doc2)
def ANSM(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    com=np.array(list(a.intersection(b)))
    sim=0
    if len(com)>0:
        sim=np.sum(np.sum(doc1[com]+doc2[com]))/(np.sum(doc1)+np.sum(doc2))
        
    return sim   
def ANSM_V1(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    SP=len(a.intersection(b))
    N=len(doc1)
    return ANSM(doc1,doc2)*((SP+1)/(N+1))
def ANSM_V2(doc1,doc2):
    return ANSM(doc1,doc2)*CSM_P2(doc1,doc2)
def ANSM_V3(doc1,doc2):
    return (ANSM(doc1,doc2)+CSM_P2(doc1,doc2))/2
def MSTB_SM(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  
    b=set(np.nonzero(doc2)[0])
    intersection=np.array(list(a.intersection(b)))
    sim=0
    if len(intersection)>0:
        ints=np.sum(np.intersect1d(doc1,doc2))
        D1=np.sum(np.unique(doc1))-ints
        D2=np.sum(np.unique(doc2))-ints
        X,Y=0,0  
        X=np.sum(doc1[intersection])
        Y=np.sum(doc2[intersection])
        a=np.array(list(a))
        b=np.array(list(b))
        Z1,Z2=0,0
        Z1=np.sum(doc1[a])
        Z2=np.sum(doc2[b]) 
        sim=((X*Y)/(Z1*Z2))*(1-((D1*D2)/(Z1*Z2)))
    return sim 
def STB_SM(doc1,doc2):
    a=set(np.nonzero(doc1)[0])  
    b=set(np.nonzero(doc2)[0])
    intersection=np.array(list(a.intersection(b)))
    comp1=a-b
    comp2=b-a   
    comp1=np.array(list(comp1))
    comp2=np.array(list(comp2))
    a=np.array(list(a))
    b=np.array(list(b))
    X,Y,D1,D2,Z1,Z2,sim=0,0,0,0,0,0,0
    if len(intersection)>0:
        X=np.sum(doc1[intersection])
        Y=np.sum(doc2[intersection])
    if len(comp1)>0:
        D1=np.sum(doc1[comp1])
    if len(comp2)>0:
        D2=np.sum(doc2[comp2])
    if len(a)>0:
        Z1=np.sum(doc1[a])
    if len(b)>0:
        Z2=np.sum(doc2[b]) 
    if Z1!=0 and Z2!=0:
        sim=((X*Y)/(Z1*Z2))*(1-((D1*D2)/(Z1*Z2)))
    return sim
def DSM(doc1,doc2,var):
    lemda=1
    a=set(np.nonzero(doc1)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2)[0])  #indices of non zero elements in doc2
    intersection=a.intersection(b) # indices where both docs have non zero elements
    union=a.union(b) # indices where either doc has non zero elements
    doc1=np.array(doc1)
    doc2=np.array(doc2)
    Nstar=0
    intersection=np.array(list(intersection))
    if (len(intersection)>0):
        term1=np.exp(-1*( doc1[intersection]-doc2[intersection] )/var[intersection])
        Nstar=sum(0.5* (1+term1)) 
    else:
        Nstar=lemda* -1 
    Nunion=len(union)
    dsm=((Nstar/Nunion)+lemda)/(1+lemda)
    return dsm
def TA(doc1,doc2):
    a=np.sqrt(doc1.dot(doc1))
    b=np.sqrt(doc2.dot(doc2))
    dot=np.dot(doc1,doc2)**2
    sim=0
    if a<=b:
        sim=dot/(a*(b**3))
    else:
        sim=dot/(b*(a**3))
    return sim
def HSD(doc1,doc2):
    sum1=0
    minimum=np.minimum(doc1, doc2)
    maximum=np.maximum(doc1, doc2)
    return 1-np.sum(1-((1+minimum)/(1+maximum)))

######################Enhnaced Measures#################################################
def EExtendedJaccard(a,b):
    vector1=set(np.where(a!=0)[0])
    vector2=set(np.where(b!=0)[0])
    dot=len(vector1.intersection(vector2))
    sum1=len(vector1)
    sum2=len(vector2)
    denom=math.sqrt(sum1)+math.sqrt(sum2)-dot
    if(denom!=0):
        return 1.0 - (float(dot)/(denom))
    else:
        return -1
    
def EPairwise(a,b):
    percentage=1 #100 percent
    K1=np.count_nonzero(a)
    K2=np.count_nonzero(b)
    k=percentage*min(K1,K2)
    setA=set(np.argpartition(a, len(a)-1 - k)[-k:])
    setB=set(np.argpartition(b, len(b) -1- k)[-k:])
    union=np.array(list(setA.union(setB)))
    elementsA=a[union]
    elementsB=b[union]
    return distance.cosine(elementsA, elementsB)

def EJS(a, b):
    p = a/ np.sum(a)
    q = b/ np.sum(b)
    m = np.add(p,q) / 2
    return (entropy(p, m) + entropy(q, m)) / 2

def EDice(doc1,doc2):
    a=set(list(np.nonzero(doc1!=0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2!=0)[0]))  #indices of non zero elements in doc2
    intr=len(a.intersection(b))
    aComp=len(a-b)
    bComp=len(b-a)
    sim=0
    if (intr+aComp+bComp)!=0:
        sim=2*(intr/(2*(intr+aComp+bComp)))
    return sim
def EExtendedDice(doc1, doc2):
    a=set(list(np.nonzero(doc1!=0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2!=0)[0]))  #indices of non zero elements in doc2
    intr=len(a.intersection(b))
    sim=(2*intr)/(len(a)**2 + len(b)**2)
    return sim
def EPDSM(doc1,doc2):
    a=set(np.nonzero(doc1!=0)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(doc2!=0)[0])  #indices of non zero elements in doc2
    intersection=np.sum(np.minimum(doc1,doc2))
    union=np.sum(np.maximum(doc1,doc2))
    PF=len(a.intersection(b))
    M=len(doc1)
    AF=M-len(a.union(b))
    psdm=(intersection/union)*((PF+1)/(M-AF+1))
    return psdm
def esmtp(doc1,doc2):
    lemda=1
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    intersection=a.intersection(b) # indices where both docs have non zero elements
    union=a.union(b) # indices where either doc has non zero elements
    l1=len(a-intersection) # doc1 !=0 and doc2=0
    l2=len(b-intersection) # doc1 =0 and doc2!=0
    Nstar=0
    intersection=np.array(list(intersection))
    if (len(intersection)>0):
        term1=np.exp(-1*np.square(( doc1[intersection]-doc2[intersection] )/var[intersection]))
        Nstar=np.sum(0.5* (1+term1)) +lemda* -1 *(l1+l2)
    else:
        Nstar=lemda* -1 *(l1+l2)   
    Nunion=len(intersection)+l1+l2
    smtp=((Nstar/Nunion)+lemda)/(1+lemda)
    return smtp
def ENSMT(doc1,doc2):
    Dij=Nij=Di=Ni=Nj=Dj=0
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    intersection=a.intersection(b)# indices where both docs have non zero elements
    a=np.array(list(a-intersection)) 
    b=np.array(list(b-intersection))
    intersection=np.array(list(intersection))
    if len(intersection)>0:
      #  prod=np.multiply(doc1[intersection],doc2[intersection])
        Dij=np.dot(doc1[intersection],doc2[intersection])#sum(prod)
    if len(a)>0:
        Di=np.sum(doc1[a])
    if len(b)>0:
        Dj=np.sum(doc2[b])
    Nij=len(intersection)
    Ni=len(a)
    Nj=len(b)
    Nsmt=0
    if (Ni*Di+Dj*Nj)!=0:
        Nsmt=(Nij*Dij)/(Ni*Di+Dj*Nj)
    return Nsmt

def EBLAB_SM(doc1,doc2):
    return 0.5*(ECSM_P1(doc1,doc2)+ECSM_P2(doc1,doc2))
    '''
    if dataset=="reuters":
        sim1=ECSMB_MinMax(doc1,doc2,alpha=0.7,Beta=0.3)#CSMB12
    else:
        sim1=ECSMB(doc1,doc2,alpha=0.9,Beta=0.1)# CSMB10
    
    return sim1
    '''
def EESTB_SM(doc1,doc2):
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    intersection= np.array(list(a.intersection(b)))
    comp1=np.array(list(a-b))
    comp2=np.array(list(b-a))
    D1=D2=sim=0                   
    if len(intersection)>0:
        X=np.sum(doc1[intersection])
        Y=np.sum(doc2[intersection])
        if len(comp1)>0:
            D1=np.sum(doc1[comp1])
        if len(comp2)>0:
            D2=np.sum(doc2[comp2])
        sim=1/(1+((D1/X)+(D2/Y)))
    return sim


def ECSM_P2(doc1,doc2):
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2NSMT
    sim=0
    if (len(a)+len(b))!=0:
        sim=(2*len(a.intersection(b)))/(len(a)+len(b))
    return sim



def EEMX1(doc1,doc2):
    a=set(np.nonzero(doc1!=0)[0])  
    b=set(np.nonzero(doc2!=0)[0])
    intersection=a.intersection(b)
    intersection=a.intersection(b)
    sim=0
    if len(intersection)>0:
        meanDiff=abs(np.mean(doc1)-np.mean(doc2))
        sim=1/(1+math.exp(-1*meanDiff))
    return sim


def EEMX2(doc1,doc2):
    a=set(np.nonzero(doc1!=0)[0])  
    b=set(np.nonzero(doc2!=0)[0])
    intersection=a.intersection(b)
    sim=0
    if len(intersection)>0:
        meanDiff=abs(np.mean(doc1)-np.mean(doc2))
        stdDiff=abs(np.std(doc1)-np.std(doc2))
        sim=1/(1+math.exp(-1*meanDiff*stdDiff))
    return sim 

def EEMX7(doc1,doc2):
    sim=EEMX1(doc1,doc2)
    if sim>0:
        sim=sim*(1-EESTB_SM(doc1,doc2))*ECSM_P2(doc1,doc2)
    return sim
def EEMX13(doc1,doc2):
    sim=EEMX2(doc1,doc2)
    if sim>0:
        sim=sim*(1-EESTB_SM(doc1,doc2))*ECSM_P2(doc1,doc2)
    return sim 

def EMSTB_SM(doc1,doc2):
    a=np.nonzero(doc1 != 0)[0]  #indices of non zero elements in doc1
    b=np.nonzero(doc2 != 0)[0]  #indices of non zero elements in doc2
    newDoc1=doc1[a]
    newDoc2=doc2[b]
    intersection=np.intersect1d(a,b)
    sim=0
    if len(intersection)>0:
        ints=np.sum(np.intersect1d(doc1,doc2))
        D1=np.sum(np.unique(newDoc1))-ints
        D2=np.sum(np.unique(newDoc2))-ints 
        X=np.sum(doc1[intersection])
        Y=np.sum(doc2[intersection])
        Z1=np.sum(newDoc1)
        Z2=np.sum(newDoc2) 
        sim=((X*Y)/(Z1*Z2))*(1-((D1*D2)/(Z1*Z2)))
    return sim 

def ECSM_P1(doc1,doc2):
    simValue=0
    N=len(doc1)#total_features;
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2NSMT
    Nab=len(a.intersection(b))
    F=len(a.union(b))-Nab
    simValue=(1-F/N)
    return simValue
def DDSMa(doc1,doc2):
    return 1-(np.sum(np.abs(doc1-doc2))/np.sum(doc1+doc2))
def DDSMb(doc1,doc2):
    return 1-(np.sum(np.square(doc1-doc2))/np.sum(np.square(doc1+doc2)))
def DDSMc(doc1,doc2):
    doc3=np.square(doc1)
    doc4=np.square(doc2)
    return 1-(np.sum(np.abs(doc3-doc4))/np.sum(np.square(doc1+doc2)))

def EEN1_DDSMa(doc1,doc2):
    return DDSMa(doc1,doc2)*ECSM_P1(doc1,doc2)
def EEN_DDSMb(doc1,doc2):
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    SF=len(a.intersection(b))
    N=len(doc1)
    return DDSMb(doc1,doc2)*(SF+1)/(N+1)
def EEN_DDSMc(doc1,doc2):
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    SF=len(a.intersection(b))
    sim=1-(DDSMc(doc1,doc2)*(SF+1)/(len(doc1)+1))
    return sim


def EANSM(doc1,doc2):
    a=np.nonzero(doc1 != 0)[0] #indices of non zero elements in doc1
    b=np.nonzero(doc2 != 0)[0]  #indices of non zero elements in doc2
    com=np.intersect1d(a,b)
    sim=0
    if len(com)>0:
        sim=np.sum(doc1[com]+doc2[com])/np.sum(doc1+doc2)
    return sim  
def EANSM_V3(doc1,doc2):
    return (EANSM(doc1,doc2)+ECSM_P2(doc1,doc2))/2

def EZSM(doc1,doc2):
    Dij=Nij=Di=Ni=Nj=Dj=0
    a=np.nonzero(doc1 != 0)[0]  #indices of non zero elements in doc1
    b=np.nonzero(doc2 != 0)[0]  #indices of non zero elements in doc2
   # sum1=np.sum(doc1[a])
    #sum2=np.sum(doc2[b])
    a=set(list(a))
    b=set(list(b)) 
    intersection=a.intersection(b)# indices where both docs have non zero elements
    intersection=np.array(list(intersection))
    Nij=len(intersection)
    if Nij>0:
        Dij=np.dot(doc1[intersection],doc2[intersection])#sum(prod)
    zsm=(Nij*Dij)/(np.sum(doc1)*np.sum(doc2))
    return zsm
def EEnhancedJaccard(a, b):#similarity
    a=set(np.nonzero(a!=0)[0])  #indices of non zero elements in doc1
    b=set(np.nonzero(b!=0)[0])  #indices of non zero elements in doc2
    union=a.union(b)
    intersection=a.intersection(b)
    sim=0
    if len(union)!=0:
        sim=len(intersection)/len(union)#similarity
    return sim
def esimIT(a,b):
    p1= np.divide(a,termOcc) 
    p2= np.divide(b,termOcc)
    minVal=np.minimum(p1,p2)
    pi=np.log((np.array(docOcc)/totalDocs))
    sIT=(2*np.sum(np.multiply(minVal,pi)))/((np.sum(np.multiply(p1,pi)))+(np.sum(np.multiply(p2,pi))))
    return sIT
def STB_SM_new(doc1,doc2):
    a=set(list(np.nonzero(doc1 != 0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2 != 0)[0]))  #indices of non zero elements in doc2
    intersection=np.array(list(a.intersection(b)))
    comp1=a-b
    comp2=b-a   
    comp1=np.array(list(comp1))
    comp2=np.array(list(comp2))
    X,Y,D1,D2,Z1,Z2,sim=0,0,0,0,0,0,0
    if len(intersection)>0:
        X=np.sum(doc1[intersection])
        Y=np.sum(doc2[intersection])
    if len(comp1)>0:
        D1=np.sum(doc1[comp1])
    if len(comp2)>0:
        D2=np.sum(doc2[comp2])
    Z1=np.sum(doc1)
    Z2=np.sum(doc2) 
    if Z1!=0 and Z2!=0:
        sim=((X*Y)/(Z1*Z2))*(1-((D1*D2)/(Z1*Z2)))
    return sim
def EBASM(doc1,doc2):
    return ECSM_P2(doc1,doc2) 
def EISC(doc1,doc2): 
    dot=np.sum(np.sqrt(np.multiply(doc1,doc2)))
    isc=dot /( math.sqrt(np.linalg.norm(doc1, ord=1))*math.sqrt(np.linalg.norm(doc2, ord=1)))
    return isc

#SP equations
def ESP(doc1,doc2):
    a=set(list(np.nonzero(doc1!=0)[0]))  #indices of non zero elements in doc1
    b=set(list(np.nonzero(doc2!=0)[0]))  #indices of non zero elements in doc2
    nonzeroTerm=list(a.intersection(b))  #indices where both docs have non zero elemments
    d1=doc1[nonzeroTerm] #doc1 values of doc1 intersection  doc2
    d2=doc2[nonzeroTerm] #doc2 values of doc1 intersection  doc2
    minArr=np.minimum(d1,d2) #minimum values  of d1 and d2
    maxArr=np.maximum(d1,d2) #maximum values  of d1 and d2
    ln=len(nonzeroTerm)
    docCount=np.zeros(ln)
    aData=allData[:,nonzeroTerm] #extracting data with only feature indices of doc1 intersection  doc2
    for ti in range(0,ln):
        docCount[ti]=count_values_in_range(aData[:,ti], minArr[ti], maxArr[ti]) #count values which lies between min and max
    #norm factor which is ist part of equation 6 in paper
    normFactor= len(a.union(b))#norm factor equal to length of doc1 union doc2
    dCount=np.array(docCount[np.nonzero(docCount!=0)[0]])# get all non zero values between min and max
    SP_val=0
    if(normFactor!=0):
        SP_val=np.sum(np.log(totalDocs/dCount)) #2nd part in equation 6 in paper
        SP_val = (SP_val/normFactor) #equation 6 in paper
    return SP_val
def ENSMT_BASM(doc1,doc2):
    return ENSMT(doc1,doc2)*BASM(doc1,doc2)
    
def PCC_BASM(doc1,doc2):
    return PCC(doc1,doc2)*BASM(doc1,doc2)
  
'''
***************************************** End of Distnace measures*******************************************

'''

def most_common(lst):
    return max(set(lst), key=lst.count)
def tokenize1(documents):
    tokens=[]
    content= documents
    tokens=(word_tokenize(content))
    tokens= [token.lower() for token in tokens ]
    tokens = [token for token in tokens if token not in stopwords]
    tokens= [token for token in tokens if token.isalpha()]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    tokens= [token for token in tokens if len(token)>3 ]
    return tokens

def display_scores(vectorizer, tfidf_result):
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
def sorted_tfs(tfs,n):
    doc,terms=tfs.shape
    ind=(np.argsort(-(np.asarray(tfs.sum(axis=0)).ravel())))
    scores=np.zeros((doc,n))
    for i in range(0,len(documents)):
        for j in range(0,n):
            if(tfs[i,ind[j]]!=0):
                scores[i,j]=tfs[i,ind[j]]
    return scores
def count_values_in_range(a, range_min, range_max):

    # "between" returns a boolean Series equivalent to left <= series <= right.
    # NA values will be treated as False.
    return ((a>=range_min) & (a<=range_max)).sum()

def dislay_tfidf(vectorizer,tfidf_result):
    print(vectorizer.get_feature_names())
    print(tfidf_result)
    
def display_scores(vectorizer, tfidf_result):
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(tfidf_result.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    return sorted_scores
def groupData(labels,categories):

    print (categories)
    groups=[]
    for i in range(0,len(categories)):
        groups.append([0,0,0])
    totalDocs=len(labels)
    print (totalDocs)
    for i in range(0,totalDocs):
        tmp=(categories.index(labels[i]))
        groups[tmp].append(i)
    for i in range(0,len(categories)):
        del (groups[i])[0:3]
    return groups

def classify(kernal='linear',termOccurance=None, docOccurance=None):
   
    kernals=['linear','poly', 'rbf', 'sigmoid']
    if kernal in kernals:
        classifier = svm.SVC(kernel=kernal).fit(train_data, train_labels)
        y_pred = classifier.predict(test_data)
    else:
        classifier= svm.SVC(kernel="precomputed")
       
        classifier = classifier.fit(GramMatrix(train_data,train_data,kernal), train_labels)
        y_pred = classifier.predict( GramMatrix(test_data,train_data,kernal))
    
   
    time2= time.time()
    
    for i in range(0,1):
        #Accuracy
    
        accuracy=accuracy_score(test_labels, y_pred)
        
        #Macro Precision
        
        macroP=precision_score(test_labels, y_pred,average='macro')
        
        #Micro Precision
        
        microP=precision_score(test_labels, y_pred,average='micro')
        
        #Macro Recall
        
        macroR=recall_score(test_labels, y_pred,average='macro')
        
        #Micro Recall
        
        microR=recall_score(test_labels, y_pred,average='micro')
        
        
        fMeasure=f1_score(test_labels, y_pred,average='macro')
       # PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="F Measure",Arr=fMeasure)
        
        m_fMeasure=f1_score(test_labels, y_pred,average='micro')
       # PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="F Measure",Arr=fMeasure)
        
        #PrintDetails(metric=metric,time=str(timedelta(seconds=(time2-time1))),measure="g Measure",Arr=gMeasure)
      
        test = label_binarize(test_labels, classes=categories)
        pred = label_binarize( y_pred, classes=categories)
        
        #roc
        roc=roc_auc_score(test, pred)
    #Accuracy
    accuracy_avg.append(accuracy)
    #Precision
    macroP_avg.append(macroP)
    microP_avg.append(microP)
    #recall
    macroR_avg.append(macroR)
    microR_avg.append(microR)
    #F measure
    macroFMeasure_avg.append(fMeasure)
    microFMeasure_avg.append( m_fMeasure)
    #Roc
    roc_avg.append(roc)
    
        
def PrintDetails(metric="smtp",time=None,measure= "Accuracy",Arr=None):
    x = PrettyTable()
    kList=[1,3,5,9,15,30,45,70,90,120]
    x.field_names = ["k","DataSet1"]
    print("Kernal",metric)
    print("time",time)
    print("measure",measure)
    tables.write("Kernal\t"+str(metric)+"\n")
    tables.write("time\t"+str(time)+"\n")
    tables.write("measure\t"+str(measure)+"\n")
    average=0
    for i in range(0,len(Arr)):
        sp="Split "+str(i+1)
        x.add_row([sp,Arr[i]]) 
        average+=Arr[i]
    x.add_row(["Average",average/len(Arr)])   
    tables.write(str(x))
    print (x)
def GramMatrix(X1, X2,metric):
    """(Pre)calculates Gram Matrix K"""
    func=globals()[metric]
    gram_matrix= pairwise_distances(X1, X2,func)
    maxval=np.nanmax(gram_matrix[gram_matrix != np.inf])
    gram_matrix[np.isnan(gram_matrix)]=maxval
    gram_matrix[np.inf==gram_matrix]=maxval
    gram_matrix[-np.inf==gram_matrix]=maxval
    return gram_matrix

dataset='webkb'
count=0
documents=[]
labels=[]
rawData=[]
token_dict = dict()
path = "webkb-data.gtar\\webkb"
loadWebKbData(path=path,documents=documents,labels=labels)
print(len(documents))
categories=list(set(labels))
totalDocs= len(documents)
print (totalDocs)
stopwords = stopwords.words('english')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
vocabulary = set()
term_docs=[]
terms=[]
totalDocs= len(documents)
print (len(documents))
print ("vectorize")
tfidf = TfidfVectorizer(tokenizer=tokenize1)
print ("fit transform")
tfs = tfidf.fit_transform(documents)
kernals=["EPDSM","PCC","STB_SM_new","esmtp","EISC","EDice","ESP","EBLAB_SM"]
# ,
arr=np.array(tfs.todense())
allData=arr
k_splits=5
kf = StratifiedKFold(n_splits=k_splits)
labels=np.array(labels)
var=0
for kernal in kernals:
    fname='table_'+dataset+'_SVM_'+kernal+'_Tfidf_5RE.txt'
    tables = open(fname, 'w')
    #tables.write("No. of features\t"+str(n)+"\n")
    time1= time.time()
    print("###############")
    accuracy_avg=[]
    macroP_avg=[]
    microP_avg=[]
    macroR_avg=[]
    microR_avg=[]
    macroFMeasure_avg=[]
    microFMeasure_avg=[]
    roc_avg=[]
    split=0
    if kernal=="smtp" or kernal=="esmtp" or kernal=="DSM":
        var1=np.var(arr,axis=0).reshape(-1,1).ravel()
        var=np.asarray(var1).ravel()
    rand_state=[0,1,10,42,30]
    for i in range(0,k_splits):
        xTrain, xTest, yTrain, yTest = train_test_split(arr, labels, test_size = 0.3, random_state = rand_state[i],stratify=labels)
        train_data=np.array(xTrain)
        test_data=np.array(xTest)
        train_labels=yTrain
        test_labels=yTest
        classify(kernal=kernal)
        #print ("nTerms",n)
        
    
    time2=time.time()
    #Accuracy
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="Accuracy",Arr=accuracy_avg)
    #Precision
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="Macro Precision",Arr=macroP_avg)
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="Micro Precision",Arr=microP_avg)
    #Recall
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="Macro Recall",Arr=macroR_avg)
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="Micro Recall",Arr=microR_avg)
    #f measure
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="F Measure",Arr=macroFMeasure_avg)

    #f measure
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="Micro F Measure",Arr=microFMeasure_avg)

    #roc
    PrintDetails(metric=kernal,time=str(timedelta(seconds=(time2-time1))),measure="ROC",Arr=roc_avg)
    tables.close()
    print("###############")



[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\Aneela\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Aneela\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Aneela\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Aneela\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


webkb-data.gtar\webkb
4199
4199
4199
vectorize
fit transform
###############
Kernal EPDSM
time 5:30:51.447751
measure Accuracy
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.7976190476190477 |
| Split 2 | 0.8031746031746032 |
| Split 3 | 0.7865079365079365 |
| Split 4 | 0.7944444444444444 |
| Split 5 | 0.8015873015873016 |
| Average | 0.7966666666666666 |
+---------+--------------------+
Kernal EPDSM
time 5:30:51.447751
measure Macro Precision
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.8917624912149199 |
| Split 2 | 0.8954011519137569 |
| Split 3 | 0.8843084153144417 |
| Split 4 | 0.8875885553288643 |
| Split 5 | 0.8917564521692876 |
| Average | 0.890163413188254  |
+---------+--------------------+
Kernal EPDSM
time 5:30:51.447751
measure Micro Precision
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+


Kernal esmtp
time 1:37:11.205378
measure Accuracy
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.915079365079365  |
| Split 2 | 0.9198412698412698 |
| Split 3 | 0.9166666666666666 |
| Split 4 | 0.9142857142857143 |
| Split 5 | 0.9198412698412698 |
| Average | 0.917142857142857  |
+---------+--------------------+
Kernal esmtp
time 1:37:11.205378
measure Macro Precision
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.9135855955842469 |
| Split 2 | 0.9292934278292266 |
| Split 3 | 0.9203810187710706 |
| Split 4 | 0.9200916934962069 |
| Split 5 | 0.9276279109828066 |
| Average | 0.9221959293327116 |
+---------+--------------------+
Kernal esmtp
time 1:37:11.205378
measure Micro Precision
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.915079365079365  |
| Split 2 | 0.9198412698412698 |
| Split 3 |

Kernal ESP
time 1 day, 2:02:53.418846
measure Accuracy
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.9111111111111111 |
| Split 2 | 0.9166666666666666 |
| Split 3 | 0.9158730158730158 |
| Split 4 | 0.9142857142857143 |
| Split 5 | 0.9182539682539682 |
| Average | 0.9152380952380952 |
+---------+--------------------+
Kernal ESP
time 1 day, 2:02:53.418846
measure Macro Precision
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.9100528653502631 |
| Split 2 | 0.9263823663160669 |
| Split 3 | 0.9220820143741792 |
| Split 4 | 0.9254526541419746 |
| Split 5 | 0.9261880878001387 |
| Average | 0.9220315975965245 |
+---------+--------------------+
Kernal ESP
time 1 day, 2:02:53.418846
measure Micro Precision
+---------+--------------------+
|    k    |      DataSet1      |
+---------+--------------------+
| Split 1 | 0.9111111111111111 |
| Split 2 | 0.916666666666666

In [2]:
sdsd

NameError: name 'sdsd' is not defined